# Determining Differentially Methylated CpG Sites

Given the genes that are differentially expressed, are there CpG sites that are differentially methylated?

In [19]:
### standard imports

import numpy as np
import pandas as pd
import re
import plotly.express as px
import matplotlib.pyplot as plt

In [20]:
# import datasets
methylation = pd.read_excel('Differential_methylated_data.xlsx')
methylation

,Index,TargetID,ProbeID_A,ProbeID_B,IDH-mut.AVG_Beta,IDH-mut.DiffScore,IDH-mut.Delta Beta,IDH-mut.Intensity,IDH-mut.Signal_A,IDH-mut.Signal_B,IDH-WT.AVG_Beta,IDH-WT.DiffScore,IDH-WT.Delta Beta,IDH-WT.Intensity,IDH-WT.Signal_A,IDH-WT.Signal_B,DMR,CHR
0,1,cg00000029_TC21,41791408,41791408,0.649410,3.685929e+02,0.185353,4844.826,1647.1740,3197.6520,0.464057,0,1,4872.125,2548.1670,2323.958,NaN,chr16
1,2,cg00000109_TC21,66725308,66725308,0.787308,9.088677e+01,0.072703,3976.348,756.1304,3220.2170,0.714605,0,1,3860.792,1050.9170,2809.875,NaN,chr3
2,3,cg00000155_BC21,87669537,87669537,0.866205,4.342945e-09,0.000454,4714.739,536.8696,4177.8700,0.865750,0,1,5059.250,589.6667,4469.583,NaN,chr7
3,4,cg00000158_BC21,47668938,47668938,0.885045,-3.437062e+00,-0.007316,5236.739,507.6956,4729.0430,0.892361,0,1,5100.375,452.4583,4647.917,NaN,chr9
4,5,cg00000165_TC21,45649248,45649248,0.198133,-3.656855e+02,-0.246057,3558.609,2836.0000,722.6087,0.444190,0,1,3763.292,2053.8750,1709.417,DMR,chr1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937050,937051,rs9363764_BC21,72799538,72799538,0.600452,3.232834e+01,0.052251,8031.739,3384.5220,4647.2170,0.548201,0,1,8000.750,3769.9170,4230.833,NaN,chr6
937051,937052,rs951295_BC11,1688858,55743931,0.512132,4.137078e+01,0.061429,13261.390,6616.7390,6644.6520,0.450703,0,1,13014.460,6971.7500,6042.708,NaN,chr15
937052,937053,rs966367_BC21,35785872,35785872,0.414911,-4.827741e+00,-0.017833,6143.521,3802.7830,2340.7390,0.432744,0,1,5773.083,3459.3330,2313.750,NaN,chr2
937053,937054,rs966367_BC22,55679490,55679490,0.413784,-8.241450e+00,-0.024545,5260.392,3252.8700,2007.5220,0.438329,0,1,4889.833,2900.2080,1989.625,NaN,chr2


In [21]:
cpg_gene = pd.read_csv('EPIC-8v2-0_A1.csv')
cpg_gene

/var/folders/_c/07yqdvgx0s9c89fyzv55m1xh0000gn/T/ipykernel_51345/4186132138.py:1: DtypeWarning: Columns (5,6,7,8,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  cpg_gene = pd.read_csv('EPIC-8v2-0_A1.csv')


,IlmnID,Name,AddressA_ID,AlleleA_ProbeSeq,AddressB_ID,AlleleB_ProbeSeq,Next_Base,Color_Channel,col,Probe_Type,...,Encode_CisReg_Site_Evid,OpenChromatin_NAME,OpenChromatin_Evidence_Count,Methyl450_Loci,Methyl27_Loci,EPICv1_Loci,Manifest_probe_match,SNP_ID,SNP_DISTANCE,SNP_MinorAlleleFrequency
0,cg25324105_BC11,cg25324105,1754126.0,ATTTATAAACTAATAACCCAAAATACATTTCCCAAAAACCTTCACA...,99753217.0,GTTTATAAACTAATAACCCGAAATACATTTCCCAAAAACCTTCGCG...,A,Red,R,cg,...,1;18;46;22,Het;Quies;TssA;TxWk;TssFlnkU;ZNF/Rpts;TssFlnk;...,261;34;540;408;120;186;50;42;12;3;7,cg25324105,NaN,cg25324105,True,NaN,NaN,NaN
1,cg25383568_TC11,cg25383568,79792482.0,AAACCAAAAAAATAACAAACTACTCACCTTATCAAAATAATTAAAA...,69667133.0,AAACCGAAAAAATAACAAACTACTCACCTTATCGAAATAATTAAAA...,C,Grn,G,cg,...,5;1;81,Quies;EnhG1;Tx;TxWk;ReprPCWk;EnhG2;ReprPC;EnhA...,689;198;536;49;138;49;1;1;2,cg25383568,NaN,cg25383568,True,rs1394936871;rs1415497516;rs1488301961;rs19238...,39;44;27;1;48;45;16;0;40;38;51;37,0;0;0;0;0;0;0;0;0;0;0;0
2,cg25455143_BC11,cg25455143,80699190.0,ATAAAAAAAAATATACAACTAATCCCCCTTTTTCCCACAACTCAAA...,7659147.0,ATAAAAAAAAATATACGACTAATCCCCCTTTTTCCCACAACTCGAA...,T,Red,R,cg,...,NaN,TssFlnkU;TssFlnkD;TssA;TssFlnk;EnhWk;EnhA1;Enh...,1215;124;240;40;16;14;10;4,cg25455143,NaN,cg25455143,True,rs1047130013;rs1051458573;rs1235675713;rs13301...,51;34;25;31;29;41;1;33;8;28;26;4;39;24;9;23,0;0;0;0;0;0;0;0.002;0.041;0;0;0;0;0;0;0
3,cg25459778_BC11,cg25459778,60797262.0,AAAAATTTAAAACAAACAACAACAACCATATTAAAAACACATCAAA...,65710482.0,AAAAATTTAAAACAAACAACGACGACCATATTAAAAACGCGTCGAA...,C,Grn,G,cg,...,67;20,TxWk;ReprPCWk;TssA;Quies;TssBiv;TssFlnk;EnhA2;...,510;3;323;268;18;457;31;4;41;4;4,cg25459778,cg25459778,cg25459778,True,rs1028295962;rs1042852826;rs1054305051;rs11639...,1;34;48;46;29;27;5;19;3;2;8;15;32;39;28;6;45;5;0,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0
4,cg25487775_BC11,cg25487775,5799427.0,AAAAACAACCTAAAAAACAACCCCCTACCCTACTCTACAAAAAAAA...,89606481.0,AAAAACAACCTAAAAAACAACCCCCTACCCTACTCTACAAAAAAAA...,A,Red,R,cg,...,NaN,EnhWk;Quies;TssFlnkU;TssBiv;TxWk;TssFlnkD;EnhA...,356;89;254;75;188;325;186;6;25;133;3;9;3;2;1;4...,cg25487775,NaN,cg25487775,True,rs1026366085;rs1044940275;rs1229309650;rs12573...,12;31;1;40;9;2;0;51;41;18;22;38,0;0;0;0;0;0;0;0;0;0;0;0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
significances = pd.read_excel('significances.xlsx')
significances

,IDH_status,p-values,Fold change,IDH_mut,IDH_mut.1,IDH_mut.2,IDH_mut.3,IDH_mut.4,IDH_mut.5,IDH_mut.6,...,IDH_wt.13,IDH_wt.14,IDH_wt.15,IDH_wt.16,IDH_wt.17,IDH_wt.18,IDH_wt.19,IDH_wt.20,IDH_wt.21,IDH_wt.22
0,Sample,p-values,log2 (IDH_mut/IDH_wt),AID51,AID29,AID30,AID31,AID32,AID33,AID34,...,AID13,AID14,AID15,AID16,AID17,AID18,AID19,AID20,AID11,AID23
1,COL4A1,0.000015,-3.200906,5.35,4.44,4.91,5.67,6.09,4.48,8.37,...,11.9,10.56,7.43,10.49,9.87,8.54,13.02,12.18,11.59,11.57
2,COL6A3,0.000174,-2.996069,2.52,2.32,3.71,2.64,3.87,2.75,5.26,...,9.66,6.81,2.36,6.18,6.94,7.93,9.35,11.03,11.59,8.56
3,CA12,0.000002,-2.709475,3.82,6.75,6.03,3.71,4,3.8,5.92,...,10.61,9.52,2.36,7,8.56,10.31,8.08,6.09,9.05,7.3
4,FABP5,0.000008,-2.608859,6.63,7.38,9.02,6.45,7.59,6.72,8.45,...,11.82,8.26,7.56,6.84,11.03,6.76,11.53,10.45,12.43,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,RICTOR,0.0,1.661848,8.66,8.91,8.14,9.21,8.53,9.03,8.49,...,6.15,6.9,7.82,7.05,5.25,7.2,4.92,6.37,5.93,6.27
745,CCL4,0.004922,1.73933,8.54,5.41,6.2,6.54,5.62,8.78,6.51,...,4.46,6.73,4.56,6.66,3.9,2.22,3.08,5.4,5.48,8.52
746,AKT3,0.0,1.740091,9.92,9.34,11.03,11.38,10.52,10.33,9.16,...,7.02,8.39,9.39,8.91,7.81,8.31,7.34,8.41,6.7,7.83
747,MAPT,0.0,1.746848,10.36,10.67,10.71,9.45,9.99,9.73,9.54,...,7.39,7.8,9.59,9.38,6.73,7.55,7.49,9.41,5.85,7.66


In [23]:
# Data cleaning

# Set the first row as column names
significances.columns = significances.iloc[0]

# Drop the first row (which is now the column names)
significances = significances[1:]

# Reset the index to start from 0
significances = significances.reset_index(drop=True)
significances

,Sample,p-values,log2 (IDH_mut/IDH_wt),AID51,AID29,AID30,AID31,AID32,AID33,AID34,...,AID13,AID14,AID15,AID16,AID17,AID18,AID19,AID20,AID11,AID23
0,COL4A1,0.000015,-3.200906,5.35,4.44,4.91,5.67,6.09,4.48,8.37,...,11.9,10.56,7.43,10.49,9.87,8.54,13.02,12.18,11.59,11.57
1,COL6A3,0.000174,-2.996069,2.52,2.32,3.71,2.64,3.87,2.75,5.26,...,9.66,6.81,2.36,6.18,6.94,7.93,9.35,11.03,11.59,8.56
2,CA12,0.000002,-2.709475,3.82,6.75,6.03,3.71,4,3.8,5.92,...,10.61,9.52,2.36,7,8.56,10.31,8.08,6.09,9.05,7.3
3,FABP5,0.000008,-2.608859,6.63,7.38,9.02,6.45,7.59,6.72,8.45,...,11.82,8.26,7.56,6.84,11.03,6.76,11.53,10.45,12.43,12
4,THBS1,0.000022,-2.587337,4.23,3.65,5.1,3.35,4,4.56,6.9,...,8.71,9.77,3.37,7.8,7.54,8.13,7.9,8.67,10.03,9.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,RICTOR,0.0,1.661848,8.66,8.91,8.14,9.21,8.53,9.03,8.49,...,6.15,6.9,7.82,7.05,5.25,7.2,4.92,6.37,5.93,6.27
744,CCL4,0.004922,1.73933,8.54,5.41,6.2,6.54,5.62,8.78,6.51,...,4.46,6.73,4.56,6.66,3.9,2.22,3.08,5.4,5.48,8.52
745,AKT3,0.0,1.740091,9.92,9.34,11.03,11.38,10.52,10.33,9.16,...,7.02,8.39,9.39,8.91,7.81,8.31,7.34,8.41,6.7,7.83
746,MAPT,0.0,1.746848,10.36,10.67,10.71,9.45,9.99,9.73,9.54,...,7.39,7.8,9.59,9.38,6.73,7.55,7.49,9.41,5.85,7.66


In [24]:
methylation.columns

Index(['Index', 'TargetID', 'ProbeID_A', 'ProbeID_B', 'IDH-mut.AVG_Beta',
       'IDH-mut.DiffScore', 'IDH-mut.Delta Beta', 'IDH-mut.Intensity',
       'IDH-mut.Signal_A', 'IDH-mut.Signal_B', 'IDH-WT.AVG_Beta',
       'IDH-WT.DiffScore', 'IDH-WT.Delta Beta', 'IDH-WT.Intensity',
       'IDH-WT.Signal_A', 'IDH-WT.Signal_B', 'DMR', 'CHR'],
      dtype='object')

In [25]:
# Data cleaning
methylation.drop(columns=['ProbeID_A', 'ProbeID_B', 'IDH-mut.AVG_Beta',
       'IDH-mut.DiffScore', 'IDH-mut.Intensity',
       'IDH-mut.Signal_A', 'IDH-mut.Signal_B', 'IDH-WT.AVG_Beta',
       'IDH-WT.DiffScore', 'IDH-WT.Delta Beta', 'IDH-WT.Intensity',
       'IDH-WT.Signal_A', 'IDH-WT.Signal_B', 'DMR', 'CHR'], axis=1, inplace=True)
methylation

,Index,TargetID,IDH-mut.Delta Beta
0,1,cg00000029_TC21,0.185353
1,2,cg00000109_TC21,0.072703
2,3,cg00000155_BC21,0.000454
3,4,cg00000158_BC21,-0.007316
4,5,cg00000165_TC21,-0.246057
...,...,...,...
937050,937051,rs9363764_BC21,0.052251
937051,937052,rs951295_BC11,0.061429
937052,937053,rs966367_BC21,-0.017833
937053,937054,rs966367_BC22,-0.024545


In [26]:
# Dropping the rows from cpg_gene that have empty values for the column UCSC_RefGene_Name 
# since those genes are in the non-coding region of the mRNA
cpg_gene['UCSC_RefGene_Name']
cpg_gene.dropna(subset=['UCSC_RefGene_Name'], inplace=True)
cpg_gene

,IlmnID,Name,AddressA_ID,AlleleA_ProbeSeq,AddressB_ID,AlleleB_ProbeSeq,Next_Base,Color_Channel,col,Probe_Type,...,Encode_CisReg_Site_Evid,OpenChromatin_NAME,OpenChromatin_Evidence_Count,Methyl450_Loci,Methyl27_Loci,EPICv1_Loci,Manifest_probe_match,SNP_ID,SNP_DISTANCE,SNP_MinorAlleleFrequency
0,cg25324105_BC11,cg25324105,1754126.0,ATTTATAAACTAATAACCCAAAATACATTTCCCAAAAACCTTCACA...,99753217.0,GTTTATAAACTAATAACCCGAAATACATTTCCCAAAAACCTTCGCG...,A,Red,R,cg,...,1;18;46;22,Het;Quies;TssA;TxWk;TssFlnkU;ZNF/Rpts;TssFlnk;...,261;34;540;408;120;186;50;42;12;3;7,cg25324105,NaN,cg25324105,True,NaN,NaN,NaN
1,cg25383568_TC11,cg25383568,79792482.0,AAACCAAAAAAATAACAAACTACTCACCTTATCAAAATAATTAAAA...,69667133.0,AAACCGAAAAAATAACAAACTACTCACCTTATCGAAATAATTAAAA...,C,Grn,G,cg,...,5;1;81,Quies;EnhG1;Tx;TxWk;ReprPCWk;EnhG2;ReprPC;EnhA...,689;198;536;49;138;49;1;1;2,cg25383568,NaN,cg25383568,True,rs1394936871;rs1415497516;rs1488301961;rs19238...,39;44;27;1;48;45;16;0;40;38;51;37,0;0;0;0;0;0;0;0;0;0;0;0
6,cg25623721_TC11,cg25623721,22762238.0,CCCATAAACACTACAAACAAAACATATACCTCCTCAAATAAAAAAA...,59724845.0,CCCATAAACACTACGAACAAAACGTATACCTCCTCAAATAAAAAAA...,C,Grn,G,cg,...,NaN,ReprPCWk;TssA;TssBiv;Quies;TssFlnk;TssFlnkU;En...,581;314;303;245;111;72;1;16;6;5;4;3;2,cg25623721,NaN,cg25623721,True,rs1040513638;rs1157200615;rs1184646146;rs12226...,24;5;16;31;14;36;20;43;35;38;4;34;1;47;10;17;9...,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0
8,cg25898577_BC11,cg25898577,43700218.0,AAACCTCTATATAACTACCAATCAACCCCTCCCCAATAACTAAATC...,89642537.0,AAACCTCTATATAACTACCAATCAACCCCTCCCCGATAACTAAATC...,A,Red,R,cg,...,1;2;14;23;47,TxWk;EnhG1;TssFlnkU;Quies;Tx;TssA;EnhWk;EnhA2;...,240;60;637;394;38;31;104;31;16;92;17;3,cg25898577,NaN,cg25898577,True,rs1042159812;rs1170083896;rs1200646945;rs12715...,23;38;41;40;0;15;25;39;16;46;1;27,0;0;0;0;0;0;0;0;0;0.002;0.007;0
9,cg25908985_BC11,cg25908985,29765438.0,TCAAACACATTAAAACTAAACTACTTATAAACAAACAACACAAATT...,76632191.0,TCGAACACATTAAAACTAAACTACTTATAAACGAACGACACGAATT...,T,Red,R,cg,...,NaN,Quies;ReprPC;EnhBiv;TssBiv;TssFlnk;ReprPCWk;Ts...,721;233;244;287;11;129;15;8;2;2;10;1,cg25908985,cg25908985,cg25908985,True,rs1201240474;rs121917856;rs1244198503;rs124596...,10;12;38;21;44;32;8;0;26;3;14;18;1;13;17;16;39...,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930113,ch.20.661824R_BC21,ch.20.661824R,51739242.0,TTCAATATAACCCCTTCTCATCTATCATAAACTAAAAAACCACTAA...,NaN,NaN,NaN,NaN,NaN,ch,...,NaN,TxWk;Tx;Quies,1007;605;51,NaN,NaN,ch.20.661824R,False,rs1261143567;rs1415961243;rs1475498805;rs15687...,11;21;18;37;15;5;46;28;3;38,0;0;0;0;0;0;0;0;0;0
930116,ch.20.741059R_TC21,ch.20.741059R,22769579.0,AAATCACTAACAAAATCCAAATCCAAATCCATATATAACAAATAAA...,NaN,NaN,NaN,NaN,NaN,ch,...,NaN,Quies;TxWk;Tx;EnhA2;ReprPCWk;EnhG1;TssFlnkD;EnhG2,269;901;470;16;2;3;1;1,NaN,NaN,ch.20.741059R,False,rs1047452773;rs1200250497;rs1203950485;rs14016...,26;24;3;2;22;44;8;51;0;12;19;30;1;4;15,0;0;0;0;0;0;0.029;0;0;0;0;0;0;0;0
930170,ch.21.551062R_BC21,ch.21.551062R,49775858.0,CTAAAATCAAACACAATTAAAACTAATAAATAACTTACAAATAAAA...,NaN,NaN,NaN,NaN,NaN,ch,...,NaN,Quies;ReprPCWk;TxWk;Tx;EnhWk;ReprPC;Het;EnhG2,764;61;619;204;8;3;2;2,NaN,NaN,ch.21.551062R,False,rs1033586582;rs1189750186;rs1200564250;rs13688...,44;19;35;9;26;16;33;21;8;28,0;0;0;0;0;0;0.001;0;0;0
930176,ch.21.664518F_BC21,ch.21.664518F,20733407.0,CTCTAAAAAAAACATTCTATATCACCAACACAAACTCTTACTAAAT...,NaN,NaN,NaN,NaN,NaN,ch,...,NaN,Quies;ReprPC;Tx;EnhA1;ReprPCWk;EnhWk;EnhBiv;En...,329;56;760;90;175;79;19;29;59;37;16;2;5;6;1,NaN,NaN,ch.21.664518F,False,rs112404719;rs1205566156;rs1270542538;rs128516...,33;20;29;38;19;28;9;37;4;11;45;2,0;0;0;0;0;0;0;0;0;0;0;0


In [27]:
# Inner merge of df1 (cpg_gene) and df2 (methylation) to keep only the rows that df1 matched df2
combined = cpg_gene.merge(methylation, how='inner', left_on='IlmnID', right_on='TargetID')
combined.rename(columns={'UCSC_RefGene_Name':'genes_of_interest'}, inplace=True)

In [28]:
# Adding columns for the p-value of the gene and log2(IDH_mut/IDH_WT) in combined
combined['p-values']=None
combined['log2(IDH_mut/IDH_WT)']=None

In [29]:
combined.columns

Index(['IlmnID', 'Name', 'AddressA_ID', 'AlleleA_ProbeSeq', 'AddressB_ID',
       'AlleleB_ProbeSeq', 'Next_Base', 'Color_Channel', 'col', 'Probe_Type',
       'Strand_FR', 'Strand_TB', 'Strand_CO', 'Infinium_Design',
       'Infinium_Design_Type', 'CHR', 'MAPINFO', 'Species', 'Genome_Build',
       'Source_Seq', 'Forward_Sequence', 'Top_Sequence', 'Rep_Num',
       'UCSC_RefGene_Group', 'genes_of_interest', 'UCSC_RefGene_Accession',
       'UCSC_CpG_Islands_Name', 'Relation_to_UCSC_CpG_Island',
       'GencodeV41_Group', 'GencodeV41_Name', 'GencodeV41_Accession',
       'Phantom5_Enhancers', 'HMM_Island', 'Regulatory_Feature_Name',
       'Regulatory_Feature_Group', '450k_Enhancer', 'DMR',
       'DNase_Hypersensitivity_NAME', 'Encode_CisReg_Site',
       'Encode_CisReg_Site_Evid', 'OpenChromatin_NAME',
       'OpenChromatin_Evidence_Count', 'Methyl450_Loci', 'Methyl27_Loci',
       'EPICv1_Loci', 'Manifest_probe_match', 'SNP_ID', 'SNP_DISTANCE',
       'SNP_MinorAlleleFrequency', 'In

In [30]:
for index in range(combined.shape[0]):
    #index = 23
    combined_gene_list = (combined['genes_of_interest'][index]).split(';')

    # Using the 'set' data type to obtain unique values from the gene_list
    unique_genes = set(combined_gene_list)

    # Converts the set back into a list
    unique_gene_list = list(unique_genes)
    #print(unique_gene_list)

    # Loops through each unique_gene in unique_gene_list. If the unique_gene is in the genes of significances,
    # set the value for p-values and log2(IDH_mut/IDH_WT)
    for unique_gene in unique_gene_list:
        sig_gene_list = significances['Sample'].tolist()
        if unique_gene in sig_gene_list:
            sig_index = sig_gene_list.index(unique_gene)
            combined['p-values'][index]=significances['p-values'][sig_index]
            combined['log2(IDH_mut/IDH_WT)'][index]=significances['log2 (IDH_mut/IDH_wt)'][sig_index]

            #for sig_index in range(significances.shape[0]):
                #if unique_gene == significances.iloc[sig_index,0]:
                    #combined['p-values'][index]=significances['p-values'][sig_index]
                    #combined['log2(IDH_mut/IDH_WT)'][index]=significances['log2 (IDH_mut/IDH_wt)'][sig_index]
combined

/var/folders/_c/07yqdvgx0s9c89fyzv55m1xh0000gn/T/ipykernel_51345/1199841270.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['p-values'][index]=significances['p-values'][sig_index]
/var/folders/_c/07yqdvgx0s9c89fyzv55m1xh0000gn/T/ipykernel_51345/1199841270.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['log2(IDH_mut/IDH_WT)'][index]=significances['log2 (IDH_mut/IDH_wt)'][sig_index]


,IlmnID,Name,AddressA_ID,AlleleA_ProbeSeq,AddressB_ID,AlleleB_ProbeSeq,Next_Base,Color_Channel,col,Probe_Type,...,EPICv1_Loci,Manifest_probe_match,SNP_ID,SNP_DISTANCE,SNP_MinorAlleleFrequency,Index,TargetID,IDH-mut.Delta Beta,p-values,log2(IDH_mut/IDH_WT)
0,cg25324105_BC11,cg25324105,1754126.0,ATTTATAAACTAATAACCCAAAATACATTTCCCAAAAACCTTCACA...,99753217.0,GTTTATAAACTAATAACCCGAAATACATTTCCCAAAAACCTTCGCG...,A,Red,R,cg,...,cg25324105,True,NaN,NaN,NaN,856780,cg25324105_BC11,-0.076107,None,None
1,cg25383568_TC11,cg25383568,79792482.0,AAACCAAAAAAATAACAAACTACTCACCTTATCAAAATAATTAAAA...,69667133.0,AAACCGAAAAAATAACAAACTACTCACCTTATCGAAATAATTAAAA...,C,Grn,G,cg,...,cg25383568,True,rs1394936871;rs1415497516;rs1488301961;rs19238...,39;44;27;1;48;45;16;0;40;38;51;37,0;0;0;0;0;0;0;0;0;0;0;0,858973,cg25383568_TC11,0.010901,None,None
2,cg25623721_TC11,cg25623721,22762238.0,CCCATAAACACTACAAACAAAACATATACCTCCTCAAATAAAAAAA...,59724845.0,CCCATAAACACTACGAACAAAACGTATACCTCCTCAAATAAAAAAA...,C,Grn,G,cg,...,cg25623721,True,rs1040513638;rs1157200615;rs1184646146;rs12226...,24;5;16;31;14;36;20;43;35;38;4;34;1;47;10;17;9...,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,866641,cg25623721_TC11,0.062056,None,None
3,cg25898577_BC11,cg25898577,43700218.0,AAACCTCTATATAACTACCAATCAACCCCTCCCCAATAACTAAATC...,89642537.0,AAACCTCTATATAACTACCAATCAACCCCTCCCCGATAACTAAATC...,A,Red,R,cg,...,cg25898577,True,rs1042159812;rs1170083896;rs1200646945;rs12715...,23;38;41;40;0;15;25;39;16;46;1;27,0;0;0;0;0;0;0;0;0;0.002;0.007;0,875230,cg25898577_BC11,0.081195,None,None
4,cg25908985_BC11,cg25908985,29765438.0,TCAAACACATTAAAACTAAACTACTTATAAACAAACAACACAAATT...,76632191.0,TCGAACACATTAAAACTAAACTACTTATAAACGAACGACACGAATT...,T,Red,R,cg,...,cg25908985,True,rs1201240474;rs121917856;rs1244198503;rs124596...,10;12;38;21;44;32;8;0;26;3;14;18;1;13;17;16;39...,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,875559,cg25908985_BC11,-0.117892,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322685,ch.20.661824R_BC21,ch.20.661824R,51739242.0,TTCAATATAACCCCTTCTCATCTATCATAAACTAAAAAACCACTAA...,NaN,NaN,NaN,NaN,NaN,ch,...,ch.20.661824R,False,rs1261143567;rs1415961243;rs1475498805;rs15687...,11;21;18;37;15;5;46;28;3;38,0;0;0;0;0;0;0;0;0;0,934902,ch.20.661824R_BC21,0.000941,None,None
322686,ch.20.741059R_TC21,ch.20.741059R,22769579.0,AAATCACTAACAAAATCCAAATCCAAATCCATATATAACAAATAAA...,NaN,NaN,NaN,NaN,NaN,ch,...,ch.20.741059R,False,rs1047452773;rs1200250497;rs1203950485;rs14016...,26;24;3;2;22;44;8;51;0;12;19;30;1;4;15,0;0;0;0;0;0;0.029;0;0;0;0;0;0;0;0,934905,ch.20.741059R_TC21,0.003108,None,None
322687,ch.21.551062R_BC21,ch.21.551062R,49775858.0,CTAAAATCAAACACAATTAAAACTAATAAATAACTTACAAATAAAA...,NaN,NaN,NaN,NaN,NaN,ch,...,ch.21.551062R,False,rs1033586582;rs1189750186;rs1200564250;rs13688...,44;19;35;9;26;16;33;21;8;28,0;0;0;0;0;0;0.001;0;0;0,934941,ch.21.551062R_BC21,-0.010316,None,None
322688,ch.21.664518F_BC21,ch.21.664518F,20733407.0,CTCTAAAAAAAACATTCTATATCACCAACACAAACTCTTACTAAAT...,NaN,NaN,NaN,NaN,NaN,ch,...,ch.21.664518F,False,rs112404719;rs1205566156;rs1270542538;rs128516...,33;20;29;38;19;28;9;37;4;11;45;2,0;0;0;0;0;0;0;0;0;0;0;0,934944,ch.21.664518F_BC21,0.001145,None,None


In [31]:
combined.dropna(subset=['p-values'], inplace=True)
combined

,IlmnID,Name,AddressA_ID,AlleleA_ProbeSeq,AddressB_ID,AlleleB_ProbeSeq,Next_Base,Color_Channel,col,Probe_Type,...,EPICv1_Loci,Manifest_probe_match,SNP_ID,SNP_DISTANCE,SNP_MinorAlleleFrequency,Index,TargetID,IDH-mut.Delta Beta,p-values,log2(IDH_mut/IDH_WT)
23,cg00347938_TC11,cg00347938,49768452.0,ACACAACAAAATCTATCCTATAACACACAAATCTAAAAATCTAAAC...,22653165.0,ACACAACGAAATCTATCCTATAACGCGCAAATCTAAAAATCTAAAC...,A,Red,R,cg,...,cg00347938,True,rs1009168079;rs1040921012;rs1158599364;rs11653...,23;8;25;42;48;26;1;22;6;29;43;51;18;38;25;41;4...,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0.003;0;0.002;0;0;...,12855,cg00347938_TC11,0.049112,0.664774,-0.145
31,cg17736920_TC11,cg17736920,64695482.0,ACAACAACTACAATCATATAACAATAAACCACTATCTACCCAACAA...,97760879.0,ACAACGACTACAATCGTATAACAATAAACCGCTATCTACCCAACGA...,A,Red,R,cg,...,cg17736920,True,rs1163470839;rs1222815704;rs1265015548;rs12825...,44;8;21;11;18;9;45;10;37;31;26;28;22;38;43;4;2...,0;0;0;0;0;0;0;0;0;0;0;0.007;0;0.608;0.007;0;0;...,620293,cg17736920_TC11,0.003533,0.003297,-1.095018
41,cg15740366_BC11,cg15740366,9690480.0,CATAACCAACCTACAACAATCAAACACTATACCACTACACCACCAT...,18719963.0,CGTAACCGACCTACAACAATCGAACGCTATACCGCTACGCCGCCGT...,T,Red,R,cg,...,cg15740366,True,rs1025902522;rs377736744,0;1,0;0,550898,cg15740366_BC11,0.136517,0.006453,-0.747736
104,cg27043630_TC11,cg27043630,87619250.0,AAACCACAACAACATACTACTAAAACTAAAACCCAACATTAATAAA...,42667360.0,AAACCGCGACGACGTACTACTAAAACTAAAACCCAACATTAATAAA...,T,Red,R,cg,...,cg27043630,True,rs1045379373;rs1302767997;rs1335711825;rs14083...,36;17;7;35;23;47;44;32;1;33;16;0;42;14;10;21;3...,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,913460,cg27043630_TC11,0.285973,0.000016,-0.987627
127,cg26440467_TC11,cg26440467,14691206.0,CTACAACTATACAACAAACACCTCTACACTAAAAACAAAAACACCC...,77670328.0,CTACAACTATACAACAAACGCCTCTACGCTAAAAACAAAAACGCCC...,C,Grn,G,cg,...,cg26440467,True,rs1000801214;rs1168589558;rs1424959855;rs14585...,24;12;30;7;4;7;8;13;25;26;28;35;36;38;39;41;47...,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,893408,cg26440467_TC11,0.033104,0.008761,0.266486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322543,ch.16.2068605F_BC21,ch.16.2068605F,43683103.0,AAACACACAAAAATAAATTATAACCTACTACACATAAAATCTAAAA...,NaN,NaN,NaN,NaN,NaN,ch,...,ch.16.2068605F,False,rs1003888752;rs1029455097;rs1054315772;rs11534...,39;24;16;33;18;6;31;8;35;40;47;30;17,0;0;0;0;0;0;0;0;0;0;0;0;0,934297,ch.16.2068605F_BC21,0.033100,0.132211,0.319746
322600,ch.7.95950239F_BO21,ch.7.95950239F,88603599.0,CAAAATAATTTCTAAAACCATATTATCTCTAAAAATCCCAACTTTC...,NaN,NaN,NaN,NaN,NaN,ch,...,ch.7.95950239F,False,rs1173702091;rs1222034184;rs1268917476;rs13574...,19;1;11;6;10;43;14;30;8;5;9,0;0;0;0;0;0;0;0;0;0;0,935795,ch.7.95950239F_BO21,0.002120,0.467914,-0.178261
322639,ch.12.921239R_BC21,ch.12.921239R,77662900.0,AAAAATAATTTTCATATTCACCATTCTTCCACCTCATTAAATTACA...,NaN,NaN,NaN,NaN,NaN,ch,...,ch.12.921239R,False,rs1181324424;rs1357111907;rs1430568858;rs57668...,47;8;19;14;16;45;37,0;0;0;0;0;0;0,933975,ch.12.921239R_BC21,0.008594,0.00232,0.30817
322666,ch.1.159606180R_BC21,ch.1.159606180R,54746401.0,AAATCTCCATTTAATTTCAACTCTCAATCTTACTCCTCAAAACAAC...,NaN,NaN,NaN,NaN,NaN,ch,...,ch.1.159606180R,False,rs1003654724;rs1014793060;rs1029302443;rs12270...,49;29;31;42;25;38;13;21;2;0,0;0;0;0;0;0;0;0;0;0,933300,ch.1.159606180R_BC21,-0.000177,0.011316,0.316848


In [32]:
# Export the DataFrame to an Excel file
combined.to_excel('Differentially Methylated CpG Sites (All Columns).xlsx', index=False)

In [33]:
combined.drop(columns=['Name', 'AddressA_ID', 'AlleleA_ProbeSeq', 'AddressB_ID',
       'AlleleB_ProbeSeq', 'Next_Base', 'Color_Channel', 'col', 'Probe_Type',
       'Strand_FR', 'Strand_TB', 'Strand_CO', 'Infinium_Design',
       'Infinium_Design_Type', 'CHR', 'MAPINFO', 'Species', 'Genome_Build',
       'Source_Seq', 'Forward_Sequence', 'Top_Sequence', 'Rep_Num',
       'UCSC_RefGene_Group', 'UCSC_RefGene_Accession',
       'GencodeV41_Group', 'GencodeV41_Name', 'GencodeV41_Accession',
       'Phantom5_Enhancers', 'HMM_Island', 'Regulatory_Feature_Name',
       'Regulatory_Feature_Group', '450k_Enhancer', 'DMR',
       'DNase_Hypersensitivity_NAME', 'Encode_CisReg_Site',
       'Encode_CisReg_Site_Evid', 'OpenChromatin_NAME',
       'OpenChromatin_Evidence_Count', 'Methyl450_Loci', 'Methyl27_Loci',
       'EPICv1_Loci', 'Manifest_probe_match', 'SNP_ID', 'SNP_DISTANCE',
       'SNP_MinorAlleleFrequency', 'Index', 'TargetID'], axis=1, inplace=True)

In [34]:
combined

,IlmnID,genes_of_interest,UCSC_CpG_Islands_Name,Relation_to_UCSC_CpG_Island,IDH-mut.Delta Beta,p-values,log2(IDH_mut/IDH_WT)
23,cg00347938_TC11,CCND1;CCND1,chr11:69654042-69654384,Island,0.049112,0.664774,-0.145
31,cg17736920_TC11,EXO1;EXO1;EXO1;EXO1,chr1:241847849-241848442,Island,0.003533,0.003297,-1.095018
41,cg15740366_BC11,IMPDH1;IMPDH1;IMPDH1;IMPDH1;IMPDH1,chr7:128405488-128406244;chr7:128409445-128410434,S_Shelf;Island,0.136517,0.006453,-0.747736
104,cg27043630_TC11,MYD88;MYD88,chr3:38136511-38137259;chr3:38138366-38139198,S_Shore;Island,0.285973,0.000016,-0.987627
127,cg26440467_TC11,TONSL;VPS28;VPS28,chr8:144427952-144428762;chr8:144429134-144429427,S_Shore;Island,0.033104,0.008761,0.266486
...,...,...,...,...,...,...,...
322543,ch.16.2068605F_BC21,MAP1LC3B;MAP1LC3B,chr16:87407781-87408065,N_Shelf,0.033100,0.132211,0.319746
322600,ch.7.95950239F_BO21,SEM1;SEM1;SEM1;SEM1;SEM1;SEM1,NaN,NaN,0.002120,0.467914,-0.178261
322639,ch.12.921239R_BC21,ARID2;ARID2,NaN,NaN,0.008594,0.00232,0.30817
322666,ch.1.159606180R_BC21,SDHC;SDHC;SDHC;SDHC;SDHC;SDHC;SDHC;SDHC;SDHC,NaN,NaN,-0.000177,0.011316,0.316848


In [35]:
# Export the DataFrame to an Excel file
combined.to_excel('Differentially Methylated CpG Sites (Selected Columns).xlsx', index=False)